
## PyZX Hybrid Mixed Quantum-Classical Circuit Demo


This demo showcases PyZX's support for hybrid quantum-classical circuits using
the 'ground' feature. This allows modeling of measurements, classical control,
and mixed quantum-classical computation within the ZX-calculus framework.

The 'ground' feature in PyZX represents classical information flow and control,
enabling the representation of:
- Quantum measurements that produce classical bits
- Classical control of quantum gates (conditional operations)
- Mixed quantum-classical algorithms
- Quantum Machine Learning Pipeline

1. Quantum Error Correction with Syndrome Extraction

Demonstrates classical syndrome processing from quantum measurements
Shows how ground vertices handle error correction decision-making
Models realistic QEC protocols with classical control loops


2. Quantum Machine Learning Pipeline

Classical feature preprocessing feeding into quantum encoding
Hybrid variational layers with classical parameter control
Classical postprocessing for gradient computation and optimization

3. Advanced Mid-Circuit Measurement Patterns

Sequential measurements with feed-forward control
Parallel measurements with joint classical processing
Correlation analysis between measurement outcomes



Let’s see how PyZX can be used to model hybrid quantum-classical circuits using its ground feature. This tutorial-style code walks through six core demonstrations using ```zx.Graph()``` objects—PyZX’s underlying graphical representation of quantum circuits, based on ZX-calculus. The key idea is that by marking certain vertices with ```ground=True```, we can distinguish classical information (like measurement outcomes or classical control signals) from purely quantum operations.

- Basic Ground Operations show how to create and inspect ground vertices.

- Measurement Circuits use ground vertices to represent quantum measurements (where quantum data collapses into classical bits).

- Conditional Gates demonstrate how classical bits can control quantum operations (e.g., applying an X gate based on a classical condition).

- Teleportation Protocol encodes a full quantum teleportation algorithm with classical communication channels modeled as ground vertices.

- Circuit Reduction highlights that ground-connected vertices affect how simplifications or adjoints of circuits are performed.

- Hybrid Algorithm Example combines classical parameters (from an optimizer) with a quantum circuit—mirroring how variational quantum algorithms work.

Mathematically, this models a hybrid computation graph: quantum operations remain unitary until measured, and classical control is handled via ground vertices that propagate classical bits through the graph. The PyZX API provides tools like is_ground, grounds, and is_hybrid to analyze and manage these mixed-mode systems. This hybrid extension enriches ZX-diagrams beyond pure quantum logic, enabling simulations of full hybrid algorithms such as variational circuits, feedback loops, and teleportation—all within one unified graph model.

In [ ]:
import pyzx as zx
from fractions import Fraction
import numpy as np
from typing import List, Tuple, Dict, Set

 In traditional ZX-calculus, graphs model quantum operations using Z and X spiders ```(nodes)``` connected by edges. This script extends that concept using "grounded" vertices, which represent classical information, such as measurement outcomes or classically controlled operations. By setting ```ground=True``` on certain vertices, PyZX can distinguish quantum behavior from classical control, enabling modeling of hybrid algorithms, such as quantum teleportation, conditional gates, and variational circuits.

Mathematically, grounding a vertex allows it to behave like a classical node in a computational graph, thus mixing quantum operations (unitary evolution) with non-unitary classical branching or feedback. The demo walks through six structured examples: (1) basic ground operations, (2) measurement circuits, (3) classically controlled gates, (4) quantum teleportation via classical bits, (5) how grounded vertices affect simplification, and (6) a hybrid variational algorithm combining classical parameter optimization with quantum circuits. Each example uses ZX-graph construction and PyZX's hybrid utilities like is_ground, ```grounds()```, and ```is_hybrid()``` to show how measurement, control, and classical logic are naturally incorporated into the quantum circuit graph structure. This demonstrates that PyZX's graphical model can support full hybrid quantum-classical computation rather than just unitary-only quantum circuits.

The functionality of PyZX’s `ground` parameter is illustrated through a simple circuit example. A single qubit is initialized, and a Hadamard operation is represented using Z/X spiders and a Hadamard edge. A Z-basis measurement is then performed, with the measurement vertex marked as *grounded*. By grounding the vertex, it is indicated that the quantum information has been measured and is now classically known. This allows the simplification routines in PyZX to treat that part of the circuit as classical, enabling further graph reductions. A classically controlled gate is conceptually placed after the measurement; while not explicitly modeled, its effect is supported through the grounding mechanism.


In [ ]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math



# Create a new ZX graph instance
g = GraphS()

# Add input and output boundary vertices for a single qubit.
in0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Input for Qubit 0
out0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=5) # Output for Qubit 0

g.set_inputs([in0])
g.set_outputs([out0])

# Apply a Hadamard gate to the qubit.
# Represented by a Z-spider connected by a Hadamard edge to an X-spider.
h_z = g.add_vertex(VertexType.Z, qubit=0, row=1, phase=0)
h_x = g.add_vertex(VertexType.X, qubit=0, row=1.1, phase=0)
g.add_edge((in0, h_z), EdgeType.SIMPLE)
g.add_edge((h_z, h_x), EdgeType.HADAMARD)
q_curr = h_x # Current point in the qubit's path

#  Perform a Z-basis measurement.
# A Z-spider acts as a Z-basis measurement when its output is later grounded or connected.
meas_q0 = g.add_vertex(VertexType.Z, qubit=0, row=2)
g.add_edge((q_curr, meas_q0), EdgeType.SIMPLE)

# Ground the measurement vertex.
#  Setting 'ground' to True for 'meas_q0'
# tells PyZX that the quantum information from this point onwards for this qubit
# is now classical. This impacts how simplification rules are applied.
g.set_ground(meas_q0, True)

# Add a conceptual "classically controlled" operation.
# In a real hybrid circuit, the classical outcome of 'meas_q0' would determine
# if this gate is applied. In ZX-calculus, grounding enables simplification that
# reflects this classical resolution.
controlled_op = g.add_vertex(VertexType.Z, qubit=0, row=3, phase=math.pi / 2) # Example Rz(pi/2) gate
g.add_edge((meas_q0, controlled_op), EdgeType.SIMPLE) # Connect from the grounded measurement

# Connect the final operational vertex to the output boundary.
g.add_edge((controlled_op, out0), EdgeType.SIMPLE)

# Visualize the initial graph.
zx.draw(g)

# Simplify the graph using ZX-calculus reduction rules.
# The 'full_reduce' function will apply a comprehensive set of rewrite rules.
# Because 'meas_q0' is grounded, the simplification process will be able to
# effectively "collapse" or remove parts of the graph related to this qubit
# once its information has been classically resolved.
zx.simplify.full_reduce(g)
zx.draw(g)

This example demonstrates a simplified Quantum Phase Estimation (QPE)
circuit, focusing on how 'ground' parameters are used to represent the classical
 measurement outcomes of the "counting" register, which would then be classically
processed to extract the phase.

The circuit components include:
 - A 'counting' register (2 qubits for simplicity).
- A 'target' qubit prepared in an eigenvector state (here, a simple |+> for a Z eigenstate).
 - Hadamard gates on the counting qubits to put them into superposition.
 - Controlled-U gates (where U is a Z rotation in this simplified case) on the target qubit,
  controlled by the counting qubits.
 - Measurements on the counting qubits, with their vertices being 'grounded' to signify
   classical readout.

In [ ]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math


# Create a new ZX graph instance
g = GraphS()

# Define input/output boundaries for counting and target qubits
# We'll use 2 qubits for the counting register (Q0, Q1) and 1 target qubit (Q2).
in_c0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Counting Qubit 0
in_c1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0) # Counting Qubit 1
in_t = g.add_vertex(VertexType.BOUNDARY, qubit=2, row=0)  # Target Qubit

out_c0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=7) # Output for Counting Qubit 0
out_c1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=7) # Output for Counting Qubit 1
out_t = g.add_vertex(VertexType.BOUNDARY, qubit=2, row=7)  # Output for Target Qubit

g.set_inputs([in_c0, in_c1, in_t])
g.set_outputs([out_c0, out_c1, out_t])

# Initialize counting qubits to |+> (Hadamard decomposition)
# Q0: Hadamard
h_c0_z = g.add_vertex(VertexType.Z, qubit=0, row=1, phase=0)
h_c0_x = g.add_vertex(VertexType.X, qubit=0, row=1.1, phase=0)
g.add_edge((in_c0, h_c0_z), EdgeType.SIMPLE)
g.add_edge((h_c0_z, h_c0_x), EdgeType.HADAMARD)
q_c0_curr = h_c0_x

# Q1: Hadamard
h_c1_z = g.add_vertex(VertexType.Z, qubit=1, row=1, phase=0)
h_c1_x = g.add_vertex(VertexType.X, qubit=1, row=1.1, phase=0)
g.add_edge((in_c1, h_c1_z), EdgeType.SIMPLE)
g.add_edge((h_c1_z, h_c1_x), EdgeType.HADAMARD)
q_c1_curr = h_c1_x

# Prepare target qubit in an eigenstate (e.g., |+> for a Z-rotation U)
# Hadamard on target qubit
h_t_z = g.add_vertex(VertexType.Z, qubit=2, row=1, phase=0)
h_t_x = g.add_vertex(VertexType.X, qubit=2, row=1.1, phase=0)
g.add_edge((in_t, h_t_z), EdgeType.SIMPLE)
g.add_edge((h_t_z, h_t_x), EdgeType.HADAMARD)
q_t_curr = h_t_x

# Controlled-U gates (U = Rz(theta) in this example, theta = pi/2)
# The phase to be estimated is theta. The controlled-U operations apply U^(2^k)
# For simplicity, we'll use CNOTs with Z-phases on the target qubit,
# acting as controlled-Rz.

# CNOT (Q0, Q2) with Z-phase on Q2 (for U^(2^0))
# This simulates controlled-Rz(theta * 2^0) = controlled-Rz(theta)
cu0_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=2)
cu0_targ_x = g.add_vertex(VertexType.X, qubit=2, row=2) # X-spider for controlled phase application
cu0_targ_z = g.add_vertex(VertexType.Z, qubit=2, row=2.1, phase=math.pi / 2) # Z-spider with phase

g.add_edge((q_c0_curr, cu0_ctrl), EdgeType.SIMPLE)
g.add_edge((q_t_curr, cu0_targ_x), EdgeType.SIMPLE)
g.add_edge((cu0_ctrl, cu0_targ_x), EdgeType.HADAMARD) # CNOT part
g.add_edge((cu0_targ_x, cu0_targ_z), EdgeType.SIMPLE) # Apply Z-phase on target after CNOT

q_c0_curr = cu0_ctrl
q_t_curr = cu0_targ_z


# CNOT (Q1, Q2) with Z-phase on Q2 (for U^(2^1))
# This simulates controlled-Rz(theta * 2^1) = controlled-Rz(2*theta)
cu1_ctrl = g.add_vertex(VertexType.Z, qubit=1, row=3)
cu1_targ_x = g.add_vertex(VertexType.X, qubit=2, row=3)
cu1_targ_z = g.add_vertex(VertexType.Z, qubit=2, row=3.1, phase=math.pi) # Phase is 2 * (pi/2) = pi

g.add_edge((q_c1_curr, cu1_ctrl), EdgeType.SIMPLE)
g.add_edge((q_t_curr, cu1_targ_x), EdgeType.SIMPLE) # Target qubit continues from previous CNOT
g.add_edge((cu1_ctrl, cu1_targ_x), EdgeType.HADAMARD) # CNOT part
g.add_edge((cu1_targ_x, cu1_targ_z), EdgeType.SIMPLE) # Apply Z-phase on target after CNOT

q_c1_curr = cu1_ctrl
q_t_curr = cu1_targ_z


# Measurements on Counting Qubits & Grounding
# After the controlled-U operations, the counting qubits are measured.
# We ground these measurement vertices, indicating their outcomes are classical.
meas_c0 = g.add_vertex(VertexType.Z, qubit=0, row=4)
meas_c1 = g.add_vertex(VertexType.Z, qubit=1, row=4)

g.add_edge((q_c0_curr, meas_c0), EdgeType.SIMPLE)
g.add_edge((q_c1_curr, meas_c1), EdgeType.SIMPLE)

# Grounding the measurement outcomes of the counting register.
# These classical bits would then be used in an Inverse Quantum Fourier Transform (IQFT)
# and classical post-processing to estimate the phase.
g.set_ground(meas_c0, True)
g.set_ground(meas_c1, True)

# Connect all current qubit paths to their respective output boundaries.
# The target qubit remains quantum until the end.
g.add_edge((q_t_curr, out_t), EdgeType.SIMPLE)
g.add_edge((meas_c0, out_c0), EdgeType.SIMPLE)
g.add_edge((meas_c1, out_c1), EdgeType.SIMPLE)
zx.draw(g)
zx.simplify.full_reduce(g)
zx.draw(g)

 This example showcases a quantum state verification process where
 an ancilla qubit is used to check a property of a primary qubit's state. The
 measurement of the ancilla qubit provides a classical "verification bit," and
we use the 'ground' parameter to model this classical readout.

The circuit involves:
- A 'primary' qubit in some initial state (e.g., |0> or a superposition).
 - An 'ancilla' qubit initialized to |0>.
- Entanglement between the primary and ancilla qubits (e.g., CNOT).
- A measurement of the ancilla qubit, which is then 'grounded'. This outcome
  is the classical verification result.
- The primary qubit continues its path (its state might be altered by the interaction).

In [ ]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math



# Create a new ZX graph instance
g = GraphS()

# Define input/output boundaries for primary and ancilla qubits
in_primary = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Primary Qubit
in_ancilla = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0) # Ancilla Qubit

out_primary = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=4) # Output for Primary Qubit
out_ancilla = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=4) # Output for Ancilla Qubit

g.set_inputs([in_primary, in_ancilla])
g.set_outputs([out_primary, out_ancilla])

# Prepare the primary qubit in a state to be verified
h_primary_z = g.add_vertex(VertexType.Z, qubit=0, row=1, phase=0)
h_primary_x = g.add_vertex(VertexType.X, qubit=0, row=1.1, phase=0)
g.add_edge((in_primary, h_primary_z), EdgeType.SIMPLE)
g.add_edge((h_primary_z, h_primary_x), EdgeType.HADAMARD)
q_primary_curr = h_primary_x
q_ancilla_curr = in_ancilla # Ancilla starts as |0>

# Entangle primary and ancilla for verification
# If primary is |0>, ancilla stays |0>. If primary is |1>, ancilla flips to |1>.
# This effectively copies a property of the primary to the ancilla.
cnot_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=2)
cnot_targ = g.add_vertex(VertexType.X, qubit=1, row=2)

g.add_edge((q_primary_curr, cnot_ctrl), EdgeType.SIMPLE)
g.add_edge((q_ancilla_curr, cnot_targ), EdgeType.SIMPLE)
g.add_edge((cnot_ctrl, cnot_targ), EdgeType.HADAMARD)

q_primary_curr = cnot_ctrl # Primary qubit continues after being a control
q_ancilla_curr = cnot_targ # Ancilla qubit has been flipped conditionally

#  Measure the ancilla qubit & Ground its outcome
# The measurement outcome of the ancilla provides the classical verification result.
# For instance, if ancilla is 0, the primary might be considered "verified" for a certain property.
meas_ancilla = g.add_vertex(VertexType.Z, qubit=1, row=3)
g.add_edge((q_ancilla_curr, meas_ancilla), EdgeType.SIMPLE)

# Grounding the ancilla measurement: This signals that its quantum information
# has been converted into a classical bit, which is our verification outcome.
g.set_ground(meas_ancilla, True)


g.add_edge((q_primary_curr, out_primary), EdgeType.SIMPLE)
g.add_edge((meas_ancilla, out_ancilla), EdgeType.SIMPLE)

# Visualize the initial complex state verification circuit
zx.draw(g)
zx.simplify.full_reduce(g)
zx.draw(g)


## PyZX Hybrid Quantum-Classical Circuit Extension
================================================

This extended demo explores additional hybrid quantum-classical circuit functionality
using the 'ground' feature in PyZX. The original framework is preserved while adding
new circuits for classical fanout, hybrid multiplexing, and measurement-controlled routing.
"""

Let’s see how to build more advanced hybrid quantum-classical circuits using PyZX. This extended example demonstrates three hybrid patterns: classical fanout, hybrid multiplexing, and measurement-controlled routing—each showing different interactions between classical control and quantum data. These circuits use ground nodes to represent classical (non-coherent) values alongside quantum wires in ZX-diagrams.

## 1. Classical Fanout Circuit
This demonstrates copying a classical bit (from a single classical input) to multiple classical outputs—a non-quantum operation allowed in classical logic but forbidden in quantum circuits (due to the no-cloning theorem).

A classical input is created with ground=True.

Two Z-spiders (also grounded) act as fan-out junctions.

Two classical outputs receive the copied bits.

Ground nodes: All except the boundary are marked classical.

ℹ️ Uses g.grounds() to count and verify grounded vertices. Useful to distinguish classical logic within hybrid systems.

## 2. Hybrid Multiplexer Circuit
Here, a classical selector bit determines which of two quantum inputs is routed forward—a simple multiplexing operation based on classical control.

A classical selector node is marked with ground=True.

Two quantum inputs enter on different wires.

A classical control Z node influences two quantum gates: mux_z and mux_x (Z and X spiders).

Both gate outputs merge at a single quantum output.

💡 Demonstrates hybrid control flow—classical bits conditionally triggering operations on quantum wires.

## 3. Measurement-Controlled Routing
This shows a quantum measurement creating a classical control, which is then used to route a quantum state to different output ports.

A quantum input goes into a measuring Z-spider (treated as classical via ground=True).

Its outcome drives a classical control Z spider.

The input is partially preserved to a qmid node (representing unmeasured path).

Depending on the control value, either out1 or out2 is selected.

🧠 This simulates a mid-circuit measurement, where the measurement outcome affects downstream circuit behavior—a common feature in NISQ-era hybrid algorithms.

In [ ]:
import pyzx as zx
from fractions import Fraction
from typing import List, Tuple, Dict, Set

class HybridCircuitExtension:
    def __init__(self):
        print("=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===\n")

    def classical_fanout(self):
        """Demonstrate classical fan-out: copying a classical bit to multiple locations."""
        print("7. Classical Fanout Circuit")
        print("-" * 30)

        g = zx.Graph()

        # Classical input bit
        classical_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0, ground=True)

        # Fan-out nodes (e.g., copy to 2 locations)
        fan1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)
        fan2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical outputs
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=2, ground=True)
        out2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=2, ground=True)

        g.add_edge((classical_input, fan1))
        g.add_edge((classical_input, fan2))
        g.add_edge((fan1, out1))
        g.add_edge((fan2, out2))

        g.set_inputs((classical_input,))
        g.set_outputs((out1, out2))

        print(f"Fanout circuit created with {g.num_vertices()} vertices")
        print(f"Ground (classical) vertices: {len(g.grounds())}")
        print(f"Graph is hybrid: {g.is_hybrid()}\n")

    def hybrid_multiplexer(self):
        """Demonstrate a hybrid multiplexer circuit: select quantum data line via classical control."""
        print("8. Hybrid Multiplexer Circuit")
        print("-" * 30)

        g = zx.Graph()

        # Classical selector (ground)
        sel = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0, ground=True)

        # Quantum inputs
        q1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)
        q2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=0)

        # Control logic to switch inputs
        ctrl = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Multiplexer logic: apply Z or X gate based on selector
        mux_z = g.add_vertex(zx.VertexType.Z, qubit=1, row=2)
        mux_x = g.add_vertex(zx.VertexType.X, qubit=2, row=2)

        # Quantum output
        out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=3, row=3)

        g.add_edge((sel, ctrl))
        g.add_edge((ctrl, mux_z))
        g.add_edge((ctrl, mux_x))
        g.add_edge((q1, mux_z))
        g.add_edge((q2, mux_x))
        g.add_edge((mux_z, out))
        g.add_edge((mux_x, out))

        g.set_inputs((sel, q1, q2))
        g.set_outputs((out,))

        print(f"Multiplexer circuit created with {g.num_vertices()} vertices")
        print(f"Hybrid: {g.is_hybrid()}")
        print(f"Ground vertices: {g.grounds()}\n")

    def measurement_controlled_routing(self):
        """Use measurement result to route a quantum state to different output paths."""
        print("9. Measurement-Controlled Routing")
        print("-" * 30)

        g = zx.Graph()

        # Quantum input
        qin = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Mid-circuit measurement (creates classical control bit)
        measure = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical control node
        ctrl = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)

        # Routing logic: if ctrl==0 -> out1; if ctrl==1 -> out2
        qmid = g.add_vertex(zx.VertexType.Z, qubit=0, row=3)
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=4)
        out2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=4)

        g.add_edge((qin, measure))
        g.add_edge((measure, ctrl))
        g.add_edge((measure, qmid))
        g.add_edge((ctrl, out1))
        g.add_edge((qmid, out2))

        g.set_inputs((qin,))
        g.set_outputs((out1, out2))

        print(f"Measurement routing circuit created")
        print(f"Hybrid: {g.is_hybrid()} | Ground vertices: {g.grounds()}\n")


if __name__ == "__main__":
    ext = HybridCircuitExtension()
    ext.classical_fanout()
    ext.hybrid_multiplexer()
    ext.measurement_controlled_routing()


=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===

7. Classical Fanout Circuit
------------------------------
Fanout circuit created with 5 vertices
Ground (classical) vertices: 5
Graph is hybrid: True

8. Hybrid Multiplexer Circuit
------------------------------
Multiplexer circuit created with 7 vertices
Hybrid: True
Ground vertices: {0, 3}

9. Measurement-Controlled Routing
------------------------------
Measurement routing circuit created
Hybrid: True | Ground vertices: {1, 2}



##  1. Ground Parameter Fundamentals
This section lays the foundation:

It creates a few graph vertices representing quantum and classical gates.

Some vertices are marked as ground, meaning they're classical.

A vertex’s ground status is examined.

Copies and adjoints of the graph are made to demonstrate that the ground status is preserved, which is key when transforming circuits (e.g., during optimization).



##  2. Quantum Error Correction with Syndrome Extraction
Here, the bit-flip code is implemented:

Data qubits are encoded.

Ancilla qubits perform syndrome extraction via entangling gates.

Measurements (as ground vertices) extract error syndromes.

A classical error decoder processes these syndromes.

Quantum correction gates apply fixes based on classical outputs.



## 3. Adaptive Quantum Algorithms
This section demonstrates dynamic circuits:

A qubit is prepared in superposition.

It undergoes measurement mid-circuit (first adaptive step).

Depending on the outcome, a branch of logic is selected:

If measured 0, do nothing.

If measured 1, apply a complex rotation sequence.

A second measurement further adapts computation.

Finally, classical postprocessing computes an output.



##  4. Quantum Machine Learning (QML)
This part builds a hybrid ML pipeline:

Classical features (e.g., image pixels) are input as ground vertices.

Preprocessing layers operate classically.

The features are encoded onto quantum qubits through parameterized gates.

Variational gates and entanglers build a quantum model.

Quantum outputs are measured and fed into:

Classical postprocessing (e.g., gradients and parameter updates).



## 5. Mid-Circuit Measurement Patterns
Two patterns are explored:

Sequential feed-forward: One qubit is measured, and based on the result, another qubit is manipulated.

Parallel measurements: Two qubits are independently measured, and their results are jointly processed in classical logic.


## 6. Ground Preservation Analysis
Here, the focus is on how well PyZX preserves classical information across graph manipulations:

It sets up a mix of quantum and classical vertices.

Edges are added across both types.

It prepares to analyze whether operations like graph copying, adjointing, or rewriting maintain the intended hybrid structure.


This example demonstrates a 3-qubit bit-flip error correction code
within the PyZX framework, specifically highlighting the use of 'ground' parameters for classical syndrome measurements.

The process involves:
1. Encoding a single logical qubit into three physical qubits.
2. Introducing a simulated bit-flip error on one of the physical qubits.
3. Performing syndrome measurements using ancilla qubits.
4. Crucially, 'grounding' the ancilla qubits after their measurement. This signifies that their quantum information has been classically observed and can be discarded, allowing for specific simplifications in the ZX diagram.
5. Showing the graph before and after simplification to observe the effect of grounding on the representation of the error correction circuit. The actual classical correction logic (applying an X gate based on syndrome) is conceptually enabled by the grounded measurements, and the graph simplification reflects this.

In [ ]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math

# Create a new ZX graph instance
g = GraphS()

# Define input and output boundaries for physical and ancilla qubits.


# Physical qubits for encoding/decoding
in_log = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Logical input
out_log = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=9) # Logical output

# Additional physical qubits used in the repetition code
q1_in = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0)
q2_in = g.add_vertex(VertexType.BOUNDARY, qubit=2, row=0)

q1_out = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=9)
q2_out = g.add_vertex(VertexType.BOUNDARY, qubit=2, row=9)

# Ancilla qubits for syndrome measurement
a0_in = g.add_vertex(VertexType.BOUNDARY, qubit=3, row=0)
a1_in = g.add_vertex(VertexType.BOUNDARY, qubit=4, row=0)

a0_out = g.add_vertex(VertexType.BOUNDARY, qubit=3, row=9)
a1_out = g.add_vertex(VertexType.BOUNDARY, qubit=4, row=9)

g.set_inputs([in_log, q1_in, q2_in, a0_in, a1_in])
g.set_outputs([out_log, q1_out, q2_out, a0_out, a1_out])

# Encoding the logical qubit (3-qubit repetition code)
# Initial state: |psi>_L = |000> + |111>
cn01_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=1) # Z-spider for CNOT control
cn01_targ = g.add_vertex(VertexType.X, qubit=1, row=1) # X-spider for CNOT target
g.add_edge((in_log, cn01_ctrl), EdgeType.SIMPLE)
g.add_edge((q1_in, cn01_targ), EdgeType.SIMPLE) # Q1 starts as |0>
g.add_edge((cn01_ctrl, cn01_targ), EdgeType.HADAMARD)

cn02_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=2)
cn02_targ = g.add_vertex(VertexType.X, qubit=2, row=2)
g.add_edge((cn01_ctrl, cn02_ctrl), EdgeType.SIMPLE) # Q0 path continues from previous CNOT
g.add_edge((q2_in, cn02_targ), EdgeType.SIMPLE) # Q2 starts as |0>
g.add_edge((cn02_ctrl, cn02_targ), EdgeType.HADAMARD)

# Connect Q1 and Q2 from their CNOT targets to the next stage
q1_encoded_out = cn01_targ
q2_encoded_out = cn02_targ
q0_encoded_out = cn02_ctrl # Q0 path continues from its second CNOT control

# Introduce a simulated error (e.g., bit-flip on Qubit 1)
# An X gate represents a bit-flip error.
error_q1 = g.add_vertex(VertexType.X, qubit=1, row=3)
g.add_edge((q1_encoded_out, error_q1), EdgeType.SIMPLE) # Apply error to Qubit 1
q1_after_error = error_q1
q0_after_error = q0_encoded_out # Q0 is unaffected
q2_after_error = q2_encoded_out # Q2 is unaffected

# Syndrome Measurement (using ancilla qubits)
# Measure s0 = Q0 XOR Q1 (for error on Q0 or Q1)
# Measure s1 = Q1 XOR Q2 (for error on Q1 or Q2)

# Syndrome Measurement 1: s0 = Q0 XOR Q1 -> Ancilla A0
synd0_ctrl_q0 = g.add_vertex(VertexType.Z, qubit=0, row=4)
synd0_ctrl_q1 = g.add_vertex(VertexType.Z, qubit=1, row=4)
synd0_targ_a0 = g.add_vertex(VertexType.X, qubit=3, row=4) # X-spider for Ancilla A0

g.add_edge((q0_after_error, synd0_ctrl_q0), EdgeType.SIMPLE)
g.add_edge((q1_after_error, synd0_ctrl_q1), EdgeType.SIMPLE)
g.add_edge((a0_in, synd0_targ_a0), EdgeType.SIMPLE) # Ancilla A0 starts as |0>

# CNOT (Q0, A0)
g.add_edge((synd0_ctrl_q0, synd0_targ_a0), EdgeType.HADAMARD)
# CNOT (Q1, A0)
g.add_edge((synd0_ctrl_q1, synd0_targ_a0), EdgeType.HADAMARD)

# Syndrome Measurement 2: s1 = Q1 XOR Q2 -> Ancilla A1
synd1_ctrl_q1 = g.add_vertex(VertexType.Z, qubit=1, row=5)
synd1_ctrl_q2 = g.add_vertex(VertexType.Z, qubit=2, row=5)
synd1_targ_a1 = g.add_vertex(VertexType.X, qubit=4, row=5) # X-spider for Ancilla A1

g.add_edge((synd0_ctrl_q1, synd1_ctrl_q1), EdgeType.SIMPLE) # Q1 path continues
g.add_edge((q2_after_error, synd1_ctrl_q2), EdgeType.SIMPLE)
g.add_edge((a1_in, synd1_targ_a1), EdgeType.SIMPLE) # Ancilla A1 starts as |0>

# CNOT (Q1, A1)
g.add_edge((synd1_ctrl_q1, synd1_targ_a1), EdgeType.HADAMARD)
# CNOT (Q2, A1)
g.add_edge((synd1_ctrl_q2, synd1_targ_a1), EdgeType.HADAMARD)

# Grounding the Ancilla Qubits (Syndrome Measurement Outcomes)
# This is where 'ground' is critical. It signals that these ancilla qubits have
# been measured and their quantum information is now classical.
# This allows for specific ZX simplification rules that "absorb" or "discard"
# the classical branches, effectively simplifying the quantum part of the circuit.
g.set_ground(synd0_targ_a0, True) # Grounding Ancilla A0 after its measurement
g.set_ground(synd1_targ_a1, True) # Grounding Ancilla A1 after its measurement

# Preserve the paths for the physical qubits (Q0, Q1, Q2) that will be corrected/decoded
q0_pre_correct = synd0_ctrl_q0
q1_pre_correct = synd1_ctrl_q1
q2_pre_correct = synd1_ctrl_q2

# Connect the physical qubits to the correction/decoding stage

q0_continue = g.add_vertex(VertexType.Z, qubit=0, row=7)
q1_continue = g.add_vertex(VertexType.Z, qubit=1, row=7)
q2_continue = g.add_vertex(VertexType.Z, qubit=2, row=7)

g.add_edge((q0_pre_correct, q0_continue), EdgeType.SIMPLE)
g.add_edge((q1_pre_correct, q1_continue), EdgeType.SIMPLE)
g.add_edge((q2_pre_correct, q2_continue), EdgeType.SIMPLE)


# Decoding the logical qubit
# This is the reverse of encoding: CNOT(Q0, Q1), CNOT(Q0, Q2)
# Here, the logical qubit will ideally be recovered on Qubit 0, if correction is successful.
cn01_decode_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=8)
cn01_decode_targ = g.add_vertex(VertexType.X, qubit=1, row=8)
g.add_edge((q0_continue, cn01_decode_ctrl), EdgeType.SIMPLE)
g.add_edge((q1_continue, cn01_decode_targ), EdgeType.SIMPLE)
g.add_edge((cn01_decode_ctrl, cn01_decode_targ), EdgeType.HADAMARD)

cn02_decode_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=8) # This will merge with previous Q0 Z-spider
cn02_decode_targ = g.add_vertex(VertexType.X, qubit=2, row=8)
g.add_edge((cn01_decode_ctrl, cn02_decode_ctrl), EdgeType.SIMPLE)
g.add_edge((q2_continue, cn02_decode_targ), EdgeType.SIMPLE)
g.add_edge((cn02_decode_ctrl, cn02_decode_targ), EdgeType.HADAMARD)


# Connect to output boundaries
g.add_edge((cn02_decode_ctrl, out_log), EdgeType.SIMPLE)
g.add_edge((cn01_decode_targ, q1_out), EdgeType.SIMPLE) # Q1 out
g.add_edge((cn02_decode_targ, q2_out), EdgeType.SIMPLE) # Q2 out

# Connect the grounded ancillas to their outputs
g.add_edge((synd0_targ_a0, a0_out), EdgeType.SIMPLE)
g.add_edge((synd1_targ_a1, a1_out), EdgeType.SIMPLE)

zx.draw(g)
zx.simplify.full_reduce(g)
zx.draw(g)

This example demonstrates a simplified Quantum Machine Learning (QML)
circuit, focusing on how 'ground' parameters are used to represent the classical
measurement outcomes that would typically feed into a classical optimization loop.

To avoid the previous `AttributeError` caused by `zx.hsimplify.from_hypergraph_form`
returning `None`, this version constructs the circuit *without* `H_BOX` vertices.
Instead, Hadamards and other operations are directly built using `Z` and `X`
spiders and `HADAMARD` edges, which are the native elements for ZX-diagrams
and are directly compatible with `zx.simplify.full_reduce`.

In QML, a Variational Quantum Eigensolver (VQE) or Quantum Neural Network (QNN)
involves:
1. Encoding classical data into quantum states.
2. Applying a Parameterized Quantum Circuit (PQC).
3. Measuring observables to obtain classical outputs.
4. Using these classical outputs in a classical optimizer to update parameters.

Here, we simulate a small two-qubit QNN evaluating a single data point.
We will particularly highlight the 'grounding' of the final measurement,
signifying its quantum information is now a classical outcome.

In [ ]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math


# Create a new ZX graph instance
g = GraphS()

# Define input and output boundaries for two qubits.
# Qubit 0 will carry the 'data' and be measured.
# Qubit 1 will act as an auxiliary qubit for entanglement in the QNN.
in0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Input for Data Qubit
in1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0) # Input for Auxiliary Qubit

out0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=6) # Output for Qubit 0's path
out1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=6) # Output for Qubit 1's path

g.set_inputs([in0, in1])
g.set_outputs([out0, out1])

# Data Encoding Layer
h_encode_z = g.add_vertex(VertexType.Z, qubit=0, row=1, phase=0) # Z-spider for Hadamard
h_encode_x = g.add_vertex(VertexType.X, qubit=0, row=1.1, phase=0) # X-spider (part of H decomposition)
g.add_edge((in0, h_encode_z), EdgeType.SIMPLE)
g.add_edge((h_encode_z, h_encode_x), EdgeType.HADAMARD) # Connect with Hadamard edge for H effect

# Parameterized Quantum Circuit (PQC) Layer (Simplified QNN)
# This represents a small, fixed-parameter quantum layer.


# Rz rotation on Qubit 0 (e.g., parameter phi_1)
rz0_1 = g.add_vertex(VertexType.Z, qubit=0, row=2, phase=math.pi / 3) # Example phase
g.add_edge((h_encode_x, rz0_1), EdgeType.SIMPLE) # Connect from the H-effect

# Rz rotation on Qubit 1 (e.g., parameter phi_2)
rz1_1 = g.add_vertex(VertexType.Z, qubit=1, row=1, phase=math.pi / 2) # Example phase
g.add_edge((in1, rz1_1), EdgeType.SIMPLE)

# CNOT gate for entanglement between Qubit 0 (control) and Qubit 1 (target)
cnot_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=3)
cnot_targ = g.add_vertex(VertexType.X, qubit=1, row=3)

g.add_edge((rz0_1, cnot_ctrl), EdgeType.SIMPLE)
g.add_edge((rz1_1, cnot_targ), EdgeType.SIMPLE)
g.add_edge((cnot_ctrl, cnot_targ), EdgeType.HADAMARD)

# Another Rz rotation on Qubit 0 after entanglement (e.g., parameter phi_3)
rz0_2 = g.add_vertex(VertexType.Z, qubit=0, row=4, phase=math.pi / 6)
g.add_edge((cnot_ctrl, rz0_2), EdgeType.SIMPLE)


# Observable Measurement and Grounding
meas_q0 = g.add_vertex(VertexType.Z, qubit=0, row=5) # Z-spider for Z-basis measurement on Qubit 0
g.add_edge((rz0_2, meas_q0), EdgeType.SIMPLE)

#  Grounding the measurement vertex.
# This signifies that the quantum information from Qubit 0 has been measured,
# yielding a classical bit. This classical outcome is then passed to the
# classical optimization routine
# Grounding allows PyZX to perform specific simplifications as this quantum path
# is effectively 'resolved' into classical information.
g.set_ground(meas_q0, True)

# Connect the auxiliary qubit (Qubit 1) to its output boundary.
# This qubit might be discarded or used for other purposes.
g.add_edge((cnot_targ, out1), EdgeType.SIMPLE)

# Connect the grounded measurement vertex to its output boundary for completeness,
# though its quantum information is effectively classically extracted.
g.add_edge((meas_q0, out0), EdgeType.SIMPLE)

zx.draw(g)
zx.simplify.full_reduce(g)
zx.draw(g)


This example extends the previous QML circuit to demonstrate advanced
mid-circuit measurement patterns within PyZX, focusing on how 'ground' parameters
facilitate the representation of classical control flow and processing.

## Key advanced patterns illustrated:
1. Sequential measurements with feed-forward control: The outcome of an early
 measurement (MCM1) conceptually influences subsequent quantum gates. We model
 this by grounding MCM1 and showing conditional paths.
2. Parallel measurements with joint classical processing: Multiple qubits are
 measured (MCM2_Q0, MCM2_Q1), and their outcomes are grounded, implying they
would be processed together classically (e.g., for syndrome extraction or feature vectors).
3. Correlation analysis between measurement outcomes: The presence of multiple
   grounded measurements across the circuit sets up a scenario where their
  classical outcomes could be correlated.

This QML circuit simulates:
 - Data Encoding
 - A first Parameterized Quantum Circuit (PQC) layer
- A Mid-Circuit Measurement (MCM1) on Qubit 0, which is grounded.
- Conditional gates on Qubit 0 based on MCM1's conceptual outcome (feed-forward).
- A second PQC layer.
- Parallel Mid-Circuit Measurements (MCM2) on Qubit 0 and Qubit 1, both grounded
  for joint classical processing and correlation analysis.

In [ ]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math



# Create a new ZX graph instance
g = GraphS()

# Define input and output boundaries for two qubits.
# Qubit 0 will carry the 'data' and undergo sequential measurements.
# Qubit 1 will act as an auxiliary qubit and be involved in parallel measurements.
in0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Input for Data Qubit
in1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0) # Input for Auxiliary Qubit

# Adjusted row for outputs to accommodate new gates/measurements
out0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=12)
out1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=12)

g.set_inputs([in0, in1])
g.set_outputs([out0, out1])

# Data Encoding Layer (Hadamard decomposed to Z-X and Hadamard edge)
# Applies an H gate to Qubit 0 to put it in superposition.
h_encode_z = g.add_vertex(VertexType.Z, qubit=0, row=1, phase=0)
h_encode_x = g.add_vertex(VertexType.X, qubit=0, row=1.1, phase=0)
g.add_edge((in0, h_encode_z), EdgeType.SIMPLE)
g.add_edge((h_encode_z, h_encode_x), EdgeType.HADAMARD)
q0_curr = h_encode_x
q1_curr = in1 # Qubit 1 remains idle for now

# First Parameterized Quantum Circuit (PQC) Layer
# Simple Rz rotation on Qubit 0 and CNOT with Qubit 1.
rz0_1 = g.add_vertex(VertexType.Z, qubit=0, row=2, phase=math.pi / 4) # Rz(pi/4)
g.add_edge((q0_curr, rz0_1), EdgeType.SIMPLE)
q0_curr = rz0_1

rz1_1 = g.add_vertex(VertexType.Z, qubit=1, row=2, phase=math.pi / 3) # Rz(pi/3)
g.add_edge((q1_curr, rz1_1), EdgeType.SIMPLE)
q1_curr = rz1_1

cnot1_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=3)
cnot1_targ = g.add_vertex(VertexType.X, qubit=1, row=3)
g.add_edge((q0_curr, cnot1_ctrl), EdgeType.SIMPLE)
g.add_edge((q1_curr, cnot1_targ), EdgeType.SIMPLE)
g.add_edge((cnot1_ctrl, cnot1_targ), EdgeType.HADAMARD)
q0_curr = cnot1_ctrl
q1_curr = cnot1_targ

#  Mid-Circuit Measurement 1 (MCM1) on Qubit 0 & Grounding (Sequential Control)
mcm1_q0 = g.add_vertex(VertexType.Z, qubit=0, row=4)
g.add_edge((q0_curr, mcm1_q0), EdgeType.SIMPLE)

# Ground MCM1: This is critical for modeling feed-forward. It tells PyZX that
# this quantum information is classically observed.
g.set_ground(mcm1_q0, True)

# Conditional Quantum Gates (Feed-Forward)
cond_gate1_q0 = g.add_vertex(VertexType.Z, qubit=0, row=5, phase=math.pi / 8)
g.add_edge((mcm1_q0, cond_gate1_q0), EdgeType.SIMPLE) # Connect from grounded measurement

#  Apply X gate if MCM1 outcome is '1' (conceptually)
cond_gate2_q0 = g.add_vertex(VertexType.X, qubit=0, row=5)
g.add_edge((mcm1_q0, cond_gate2_q0), EdgeType.SIMPLE)

# Re-converge the paths for Qubit 0. This spider represents the state of Qubit 0
# after one of the conditional gates has been applied.
q0_reconverge = g.add_vertex(VertexType.Z, qubit=0, row=6)
g.add_edge((cond_gate1_q0, q0_reconverge), EdgeType.SIMPLE)
g.add_edge((cond_gate2_q0, q0_reconverge), EdgeType.SIMPLE)
q0_curr = q0_reconverge

# Second Parameterized Quantum Circuit (PQC) Layer
# Another CNOT gate for more entanglement, involving Qubit 0 (now potentially modified
# by feed-forward) and Qubit 1.
cnot2_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=7)
cnot2_targ = g.add_vertex(VertexType.X, qubit=1, row=7)
g.add_edge((q0_curr, cnot2_ctrl), EdgeType.SIMPLE)
g.add_edge((q1_curr, cnot2_targ), EdgeType.SIMPLE) # Qubit 1 continues its path
g.add_edge((cnot2_ctrl, cnot2_targ), EdgeType.HADAMARD)
q0_curr = cnot2_ctrl
q1_curr = cnot2_targ

# Parallel Mid-Circuit Measurements (MCM2) & Joint Classical Processing
# Measure both Qubit 0 and Qubit 1. Their outcomes are grounded, indicating they
# are processed jointly by classical logic (e.g., for feature extraction for a classifier,
# or for analyzing quantum correlations).
mcm2_q0 = g.add_vertex(VertexType.Z, qubit=0, row=8)
mcm2_q1 = g.add_vertex(VertexType.Z, qubit=1, row=8)

g.add_edge((q0_curr, mcm2_q0), EdgeType.SIMPLE)
g.add_edge((q1_curr, mcm2_q1), EdgeType.SIMPLE)

# Ground both MCM2 measurements. This implies classical processing of both outcomes.
g.set_ground(mcm2_q0, True)
g.set_ground(mcm2_q1, True)

# Note: The 'grounded' vertices represent the points where quantum information
# becomes classical. Subsequent classical logic would take these two outcomes
# (e.g., bit values '0' or '1' from each measurement) and perform joint processing
# or correlation analysis. The ZX diagram will simplify based on this classicality.

#  Final connection to outputs (from grounded measurements)
# Connect the grounded measurement vertices to their respective output boundaries.
# The quantum information is already classically resolved at this point.
g.add_edge((mcm2_q0, out0), EdgeType.SIMPLE)
g.add_edge((mcm2_q1, out1), EdgeType.SIMPLE)

zx.draw(g)
zx.simplify.full_reduce(g)
zx.draw(g)